In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import permutations
import re
from sympy import symbols, solve, And, Interval, reduce_inequalities

In [3]:
basic_support = 'ppqqrr'
perms = [''.join(p) for p in permutations(basic_support)]
perms = [x for x in set(perms) if x.startswith('pq') or x.startswith('ppq')]
len(perms), perms

(15,
 ['pqqprr',
  'ppqrqr',
  'pqrpqr',
  'pqrrqp',
  'ppqrrq',
  'pqprrq',
  'pqrrpq',
  'pqqrpr',
  'pqprqr',
  'pqrprq',
  'pqqrrp',
  'pqrqpr',
  'pqpqrr',
  'pqrqrp',
  'ppqqrr'])

In [4]:
pre_orderings = []
for supp in perms:
    for div in range(32):
        divisions = f'{div:05b}'
        l = list(supp)
        for i in range(5):
            l.insert(2*i+1, '|' if divisions[i]=='1' else '')
        pre_orderings.append(''.join(l))
len(pre_orderings)

480

In [5]:
orderings = []
for ord in pre_orderings:
    buckets = ord.split('|')
    buckets_new = [''.join(sorted(x)) for x in buckets]
    orderings.append('|'.join(buckets_new))
len(set(orderings))

208

In [ ]:
dict_ord = {o:{} for o in set(orderings)}
# Probability of support size of 1 for a lottery? (i.e. choose from n^2 or n(n-1))
pos_prob_counter = 0
for order in dict_ord:
    dict_ord[order]["has_pos_prob"] = not ('pp' in order or 'qq' in order or 'rr' in order)
    pos_prob_counter += not ('pp' in order or 'qq' in order or 'rr' in order)
pos_prob_counter, dict_ord

(104,
 {'p|qr|pqr': {'has_pos_prob': True},
  'pq|q|pr|r': {'has_pos_prob': True},
  'pp|qr|q|r': {'has_pos_prob': False},
  'p|q|p|r|r|q': {'has_pos_prob': True},
  'p|p|qr|qr': {'has_pos_prob': True},
  'p|qq|p|rr': {'has_pos_prob': False},
  'pqr|r|q|p': {'has_pos_prob': True},
  'p|qq|r|r|p': {'has_pos_prob': False},
  'p|q|r|q|r|p': {'has_pos_prob': True},
  'pqq|pr|r': {'has_pos_prob': False},
  'p|q|prr|q': {'has_pos_prob': False},
  'pqq|p|rr': {'has_pos_prob': False},
  'pqqr|pr': {'has_pos_prob': False},
  'pq|pr|r|q': {'has_pos_prob': True},
  'p|p|q|qrr': {'has_pos_prob': False},
  'ppqr|qr': {'has_pos_prob': False},
  'ppqq|rr': {'has_pos_prob': False},
  'p|qqr|p|r': {'has_pos_prob': False},
  'pqr|q|p|r': {'has_pos_prob': True},
  'p|qr|pq|r': {'has_pos_prob': True},
  'ppq|r|qr': {'has_pos_prob': False},
  'p|qr|r|pq': {'has_pos_prob': True},
  'pqr|pr|q': {'has_pos_prob': True},
  'p|q|r|q|p|r': {'has_pos_prob': True},
  'p|q|r|p|q|r': {'has_pos_prob': True},
  'p|q|pq

In [17]:
for order in dict_ord:
    p_pref_to_q = re.search('p[r\\|]*p[r\\|]*q[r\\|]*q[r\\|]*', order)
    dict_ord[order]["p_pref_to_q"] = False if p_pref_to_q is None else True
    p_pref_to_r = re.search('p[q\\|]*p[q\\|]*r[q\\|]*r[q\\|]*', order)
    dict_ord[order]["p_pref_to_r"] = False if p_pref_to_r is None else True
    q_pref_to_r = re.search('q[p\\|]*q[p\\|]*r[p\\|]*r[p\\|]*', order)
    dict_ord[order]["q_pref_to_r"] = False if q_pref_to_r is None else True
dict_ord

{'p|qr|pqr': {'has_pos_prob': True,
  'p_pref_to_q': False,
  'p_pref_to_r': False,
  'q_pref_to_r': False},
 'pq|q|pr|r': {'has_pos_prob': True,
  'p_pref_to_q': False,
  'p_pref_to_r': True,
  'q_pref_to_r': True},
 'pp|qr|q|r': {'has_pos_prob': False,
  'p_pref_to_q': True,
  'p_pref_to_r': True,
  'q_pref_to_r': False},
 'p|q|p|r|r|q': {'has_pos_prob': True,
  'p_pref_to_q': False,
  'p_pref_to_r': True,
  'q_pref_to_r': False},
 'p|p|qr|qr': {'has_pos_prob': True,
  'p_pref_to_q': True,
  'p_pref_to_r': True,
  'q_pref_to_r': False},
 'p|qq|p|rr': {'has_pos_prob': False,
  'p_pref_to_q': False,
  'p_pref_to_r': True,
  'q_pref_to_r': True},
 'pqr|r|q|p': {'has_pos_prob': True,
  'p_pref_to_q': False,
  'p_pref_to_r': False,
  'q_pref_to_r': False},
 'p|qq|r|r|p': {'has_pos_prob': False,
  'p_pref_to_q': False,
  'p_pref_to_r': False,
  'q_pref_to_r': True},
 'p|q|r|q|r|p': {'has_pos_prob': True,
  'p_pref_to_q': False,
  'p_pref_to_r': False,
  'q_pref_to_r': False},
 'pqq|pr|r': 

In [11]:
p, q, r = symbols('p q r', real=True)
ineq1 = p - (1 - p) * (2 * q - 1) > 0
ineq2 = q - (1 - q) * (2 * r - 1) > 0
ineq3 = (1 - p) * (2 * r - 1) - p > 0
# domain = Interval(0, 1)

system = [ineq1, ineq2, ineq3]
print("Solving the system of inequalities")
solution = reduce_inequalities(system, [p])
print("Solution:", solution)

Solving the system of inequalities


NotImplementedError: 
inequality has more than one symbol of interest.